In [1]:
# check python version
from platform import python_version

print(python_version())

3.8.16


# Install Dependency

In [2]:
# install dependencies
!pip3 install polygon-api-client --user
!pip3 install sqlalchemy --user
!pip3 install pandas --user
!pip3 install numpy --user
!pip3 install scikit-learn --user
!pip3 install pycaret --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
# Import required libraries
import datetime
import time
from polygon import RESTClient
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor
import sqlite3
import numpy as np

# 10 blocks of code defining the classes for storing data

In [4]:
# The following 10 blocks of code define the classes for storing the the return data, for each
# currency pair.

# Define the AUDUSD_return class - each instance will store one row from the dataframe
class AUDUSD_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if AUDUSD_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - AUDUSD_return.last_price) / AUDUSD_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            AUDUSD_return.run_sum = 0
        else:
            # Increment the counter
            if AUDUSD_return.num < 5:
                AUDUSD_return.num += 1
            AUDUSD_return.run_sum += hist_return
        AUDUSD_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            AUDUSD_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            AUDUSD_return.run_sum -= pop_value
            avg = AUDUSD_return.run_sum / (AUDUSD_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(AUDUSD_return.run_squared_sum / (AUDUSD_return.num))
            self.std_return = std
            AUDUSD_return.run_sum_of_std += std
            AUDUSD_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            AUDUSD_return.run_sum_of_std -= pop_value
            avg_std = AUDUSD_return.run_sum_of_std / (AUDUSD_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [5]:
# Define the GBPEUR_return class - each instance will store one row from the dataframe
class GBPEUR_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if GBPEUR_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - GBPEUR_return.last_price) / GBPEUR_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            GBPEUR_return.run_sum = 0
        else:
            # Increment the counter
            if GBPEUR_return.num < 5:
                GBPEUR_return.num += 1
            GBPEUR_return.run_sum += hist_return
        GBPEUR_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            GBPEUR_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            GBPEUR_return.run_sum -= pop_value
            avg = GBPEUR_return.run_sum / (GBPEUR_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(GBPEUR_return.run_squared_sum / (GBPEUR_return.num))
            self.std_return = std
            GBPEUR_return.run_sum_of_std += std
            GBPEUR_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            GBPEUR_return.run_sum_of_std -= pop_value
            avg_std = GBPEUR_return.run_sum_of_std / (GBPEUR_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [6]:
# Define the USDCAD_return class - each instance will store one row from the dataframe
class USDCAD_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDCAD_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDCAD_return.last_price) / USDCAD_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDCAD_return.run_sum = 0
        else:
            # Increment the counter
            if USDCAD_return.num < 5:
                USDCAD_return.num += 1
            USDCAD_return.run_sum += hist_return
        USDCAD_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDCAD_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDCAD_return.run_sum -= pop_value
            avg = USDCAD_return.run_sum / (USDCAD_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDCAD_return.run_squared_sum / (USDCAD_return.num))
            self.std_return = std
            USDCAD_return.run_sum_of_std += std
            USDCAD_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDCAD_return.run_sum_of_std -= pop_value
            avg_std = USDCAD_return.run_sum_of_std / (USDCAD_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [7]:
# Define the USDJPY_return class - each instance will store one row from the dataframe
class USDJPY_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDJPY_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDJPY_return.last_price) / USDJPY_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDJPY_return.run_sum = 0
        else:
            # Increment the counter
            if USDJPY_return.num < 5:
                USDJPY_return.num += 1
            USDJPY_return.run_sum += hist_return
        USDJPY_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDJPY_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDJPY_return.run_sum -= pop_value
            avg = USDJPY_return.run_sum / (USDJPY_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDJPY_return.run_squared_sum / (USDJPY_return.num))
            self.std_return = std
            USDJPY_return.run_sum_of_std += std
            USDJPY_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDJPY_return.run_sum_of_std -= pop_value
            avg_std = USDJPY_return.run_sum_of_std / (USDJPY_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [8]:
# Define the USDMXN_return class - each instance will store one row from the dataframe
class USDMXN_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDMXN_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDMXN_return.last_price) / USDMXN_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDMXN_return.run_sum = 0
        else:
            # Increment the counter
            if USDMXN_return.num < 5:
                USDMXN_return.num += 1
            USDMXN_return.run_sum += hist_return
        USDMXN_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDMXN_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDMXN_return.run_sum -= pop_value
            avg = USDMXN_return.run_sum / (USDMXN_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDMXN_return.run_squared_sum / (USDMXN_return.num))
            self.std_return = std
            USDMXN_return.run_sum_of_std += std
            USDMXN_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDMXN_return.run_sum_of_std -= pop_value
            avg_std = USDMXN_return.run_sum_of_std / (USDMXN_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [9]:
# Define the EURUSD_return class - each instance will store one row from the dataframe
class EURUSD_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if EURUSD_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - EURUSD_return.last_price) / EURUSD_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            EURUSD_return.run_sum = 0
        else:
            # Increment the counter
            if EURUSD_return.num < 5:
                EURUSD_return.num += 1
            EURUSD_return.run_sum += hist_return
        EURUSD_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            EURUSD_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            EURUSD_return.run_sum -= pop_value
            avg = EURUSD_return.run_sum / (EURUSD_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(EURUSD_return.run_squared_sum / (EURUSD_return.num))
            self.std_return = std
            EURUSD_return.run_sum_of_std += std
            EURUSD_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            EURUSD_return.run_sum_of_std -= pop_value
            avg_std = EURUSD_return.run_sum_of_std / (EURUSD_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [10]:
# Define the USDCNY_return class - each instance will store one row from the dataframe
class USDCNY_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDCNY_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDCNY_return.last_price) / USDCNY_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDCNY_return.run_sum = 0
        else:
            # Increment the counter
            if USDCNY_return.num < 5:
                USDCNY_return.num += 1
            USDCNY_return.run_sum += hist_return
        USDCNY_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDCNY_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDCNY_return.run_sum -= pop_value
            avg = USDCNY_return.run_sum / (USDCNY_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDCNY_return.run_squared_sum / (USDCNY_return.num))
            self.std_return = std
            USDCNY_return.run_sum_of_std += std
            USDCNY_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDCNY_return.run_sum_of_std -= pop_value
            avg_std = USDCNY_return.run_sum_of_std / (USDCNY_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [11]:
# Define the USDCZK_return class - each instance will store one row from the dataframe
class USDCZK_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDCZK_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDCZK_return.last_price) / USDCZK_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDCZK_return.run_sum = 0
        else:
            # Increment the counter
            if USDCZK_return.num < 5:
                USDCZK_return.num += 1
            USDCZK_return.run_sum += hist_return
        USDCZK_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDCZK_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDCZK_return.run_sum -= pop_value
            avg = USDCZK_return.run_sum / (USDCZK_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDCZK_return.run_squared_sum / (USDCZK_return.num))
            self.std_return = std
            USDCZK_return.run_sum_of_std += std
            USDCZK_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDCZK_return.run_sum_of_std -= pop_value
            avg_std = USDCZK_return.run_sum_of_std / (USDCZK_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [12]:
# Define the USDPLN_return class - each instance will store one row from the dataframe
class USDPLN_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDPLN_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDPLN_return.last_price) / USDPLN_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDPLN_return.run_sum = 0
        else:
            # Increment the counter
            if USDPLN_return.num < 5:
                USDPLN_return.num += 1
            USDPLN_return.run_sum += hist_return
        USDPLN_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDPLN_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDPLN_return.run_sum -= pop_value
            avg = USDPLN_return.run_sum / (USDPLN_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDPLN_return.run_squared_sum / (USDPLN_return.num))
            self.std_return = std
            USDPLN_return.run_sum_of_std += std
            USDPLN_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDPLN_return.run_sum_of_std -= pop_value
            avg_std = USDPLN_return.run_sum_of_std / (USDPLN_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [13]:
# Define the USDINR_return class - each instance will store one row from the dataframe
class USDINR_return(object):
    # Variable to store the total number of instantiated objects in this class
    num = 0
    # Variable to store the running sum of the return
    run_sum = 0
    run_squared_sum = 0
    run_sum_of_std = 0
    last_price = -1

    # Init all the necessary variables when instantiating the class
    def __init__(self, tick_time, avg_price):

        # Store each column value into a variable in the class instance
        self.tick_time = tick_time
        #self.price = avg_price

        if USDINR_return.last_price == -1:
            hist_return = float('NaN')
        else:
            hist_return = (avg_price - USDINR_return.last_price) / USDINR_return.last_price

        self.hist_return = hist_return
        if isnan(hist_return):
            USDINR_return.run_sum = 0
        else:
            # Increment the counter
            if USDINR_return.num < 5:
                USDINR_return.num += 1
            USDINR_return.run_sum += hist_return
        USDINR_return.last_price = avg_price

    def add_to_running_squared_sum(self, avg):
        if isnan(self.hist_return) == False:
            USDINR_return.run_squared_sum += (self.hist_return - avg) ** 2

    def get_avg(self, pop_value):
        if isnan(self.hist_return) == False:
            USDINR_return.run_sum -= pop_value
            avg = USDINR_return.run_sum / (USDINR_return.num)
            self.avg_return = avg
            return avg

    def get_std(self):
        if isnan(self.hist_return) == False:
            std = sqrt(USDINR_return.run_squared_sum / (USDINR_return.num))
            self.std_return = std
            USDINR_return.run_sum_of_std += std
            USDINR_return.run_squared_sum = 0
            return std

    def get_avg_std(self, pop_value):
        if isnan(self.hist_return) == False:
            USDINR_return.run_sum_of_std -= pop_value
            avg_std = USDINR_return.run_sum_of_std / (USDINR_return.num)
            self.avg_of_std_return = avg_std
            return avg_std

In [14]:
# We can buy, sell, or do nothing each time we make a decision.
# This class defies a nobject for keeping track of our current investments/profits for each currency pair
class portfolio(object):
    def __init__(self, from_, to):
        # Initialize the 'From' currency amont to 1
        self.amount = 1
        self.curr2 = 0
        self.from_ = from_
        self.to = to
        # We want to keep track of state, to see what our next trade should be
        self.Prev_Action_was_Buy = False

    # This defines a function to buy the 'To' currency. It will always buy the max amount, in whole number
    # increments
    def buy_curr(self, price):
        if self.amount >= 1:
            num_to_buy = floor(self.amount)
            self.amount -= num_to_buy
            self.Prev_Action_was_Buy = True
            self.curr2 += num_to_buy * price
            print(
                "Bought %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (
                num_to_buy, self.to, self.from_, (self.amount - 1)))
        else:
            print("There was not enough of the original currency (%s) to make another buy." % self.from_)
    # This defines a function to sell the 'To' currency. It will always sell the max amount, in a whole number
    # increments
    def sell_curr(self, price):
        if self.curr2 >= 1:
            num_to_sell = floor(self.curr2)
            self.amount += num_to_sell * (1 / price)
            self.Prev_Action_was_Buy = False
            self.curr2 -= num_to_sell
            print(
                "Sold %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (
                num_to_sell, self.to, self.from_, (self.amount - 1)))
        else:
            print("There was not enough of the target currency (%s) to make another sell." % self.to)

# Train Model

In [15]:
import pandas as pd

In [16]:
# read data from csv
columns = ["time","mean", "std", "max", "min", "vol", "fd","r"]
AUDUSD_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/AUDUSD_agg.csv', names=columns, na_values='?', header=None,dtype={"time":'str',"mean":'float64', "std":'float64', "max":'float64', "min":'float64', "vol":'float64', "fd":'float64',"r":'float64'})
EURUSD_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/EURUSD_agg.csv', names=columns, na_values='?', header=None)
GBPEUR_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/GBPEUR_agg.csv', names=columns, na_values='?', header=None)
USDCAD_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDCAD_agg.csv', names=columns, na_values='?', header=None)
USDCNY_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDCNY_agg.csv', names=columns, na_values='?', header=None)
USDCZK_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDCZK_agg.csv', names=columns, na_values='?', header=None)
USDINR_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDINR_agg.csv', names=columns, na_values='?', header=None)
USDJPY_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDJPY_agg.csv', names=columns, na_values='?', header=None)
USDMXN_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDMXN_agg.csv', names=columns, na_values='?', header=None)
USDPLN_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-ten-hours/USDPLN_agg.csv', names=columns, na_values='?', header=None)

In [17]:
# match the currency pairs and data frame
currency_pairs_dict = {"AUDUSD": AUDUSD_df, "EURUSD": EURUSD_df, "GBPEUR": GBPEUR_df, "USDCAD": USDCAD_df,
                       "USDCNY": USDCNY_df, "USDCZK": USDCZK_df, "USDINR": USDINR_df, "USDJPY": USDJPY_df,
                       "USDMXN": USDMXN_df, "USDPLN": USDPLN_df}

In [64]:
# classify the data by vol/FD
# set them as 1=low, 2=medium, 3=high
# first bar=top 33
# second bar=top=67
def classify(original_df, column_name,first_bar=33, second_bar=67):
    df=original_df.copy(deep=True)
    # sort by vol
    df=df.sort_values(by=[column_name])
    # first_bar_dict[column_name]
    for i in range(len(df)):
        if i < first_bar:
            df.at[i,column_name] = 1
        elif i < second_bar:
            df.at[i,column_name] = 2
        else:
            df.at[i,column_name] = 3
    return df

In [65]:
formatted_currency_pairs_dict={}
# classify all the fds
for key, df in currency_pairs_dict.items():
    # classify by fd
    df=classify(df,"fd")
    # classify by vol
    df=classify(df,"vol")
    # show the results to check
    print(df)
    formatted_currency_pairs_dict[key]=df

                   time      mean       std       max       min  vol  fd  \
0   2022-12-05 15:30:54  0.669145  0.000221  0.669505  0.668700    1   1   
3   2022-12-05 16:01:05  0.669215  0.000064  0.669400  0.669000    1   1   
38  2022-12-05 21:18:27  0.672492  0.000225  0.672900  0.671900    2   2   
39  2022-12-05 21:27:24  0.672297  0.000246  0.672600  0.671665    2   2   
2   2022-12-05 15:52:24  0.669124  0.000078  0.669415  0.668900    1   1   
..                  ...       ...       ...       ...       ...  ...  ..   
11  2022-12-05 17:14:14  0.670027  0.000298  0.672625  0.669700    1   1   
16  2022-12-05 17:58:54  0.670178  0.000126  0.671700  0.669940    1   1   
32  2022-12-05 20:20:43  0.671676  0.000385  0.672500  0.671085    1   1   
59  2022-12-06 00:23:54  0.672865  0.000217  0.673200  0.672000    2   2   
49  2022-12-05 22:55:28  0.672830  0.000288  0.673300  0.671800    2   2   

           r  
0   0.000000  
3   0.000137  
38  0.000580  
39 -0.000289  
2   0.000022

## Call prcaret to train model

In [66]:
from pycaret import regression

In [67]:
# extract features we want to use to train the model
def extract_features(df):
    df=df[["mean","fd","vol","r"]]
    return df

In [68]:
# train model for all currency df
def train_regression_model(df, target):
    regression.setup(data=df,target=target,fold_shuffle=True)
    best=regression.compare_models()
    regression.evaluate_model(best)
    return best

In [69]:
currency_model_dict={}
for curreny, df in formatted_currency_pairs_dict.items():
    # extract feature
    df=extract_features(df)
    # print(df)
    currency_model_dict[curreny]=train_regression_model(df,'r')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lar,Least Angle Regression,0.0003,0.0,0.0005,-0.2897,0.0004,3.2069,0.013
ridge,Ridge Regression,0.0003,0.0,0.0005,-0.2923,0.0004,3.8943,0.012
br,Bayesian Ridge,0.0003,0.0,0.0005,-0.2932,0.0004,3.9741,0.019
par,Passive Aggressive Regressor,0.0003,0.0,0.0005,-0.2936,0.0005,1.0000,0.025
lr,Linear Regression,0.0003,0.0,0.0005,-0.3163,0.0004,5.8617,0.013
lightgbm,Light Gradient Boosting Machine,0.0004,0.0,0.0005,-0.3349,0.0004,3.2397,0.021
huber,Huber Regressor,0.0003,0.0,0.0005,-0.3396,0.0004,4.2657,0.045
en,Elastic Net,0.0004,0.0,0.0005,-0.3538,0.0005,1.7547,0.015
llar,Lasso Least Angle Regression,0.0004,0.0,0.0005,-0.3538,0.0005,1.7547,0.015
lasso,Lasso Regression,0.0004,0.0,0.0005,-0.3538,0.0005,1.7547,0.015


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:Lars(copy_X=True, eps=2.220446049250313e-16, fit_intercept=True, fit_path=True,
     jitter=None, n_nonzero_coefs=500, normalize=True, precompute='auto',
     random_state=499, verbose=False)
INFO:logs:compare_models() succesfully completed......................................
INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=Lars(copy_X=True, eps=2.220446049250313e-16, fit_intercept=True, fit_path=True,
     jitter=None, n_nonzero_coefs=500, normalize=True, precompute='auto',
     random_state=499, verbose=False), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [70]:
# Read one hour data to test
columns = ["time","mean", "std", "max", "min", "vol", "fd","r"]
AUDUSD_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/AUDUSD_agg.csv', names=columns, na_values='?', header=None,dtype={"time":'str',"mean":'float64', "std":'float64', "max":'float64', "min":'float64', "vol":'float64', "fd":'float64',"r":'float64'})
EURUSD_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/EURUSD_agg.csv', names=columns, na_values='?', header=None)
GBPEUR_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/GBPEUR_agg.csv', names=columns, na_values='?', header=None)
USDCAD_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDCAD_agg.csv', names=columns, na_values='?', header=None)
USDCNY_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDCNY_agg.csv', names=columns, na_values='?', header=None)
USDCZK_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDCZK_agg.csv', names=columns, na_values='?', header=None)
USDINR_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDINR_agg.csv', names=columns, na_values='?', header=None)
USDJPY_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDJPY_agg.csv', names=columns, na_values='?', header=None)
USDMXN_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDMXN_agg.csv', names=columns, na_values='?', header=None)
USDPLN_test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/data-engineering-hw5/data-one-hour/USDPLN_agg.csv', names=columns, na_values='?', header=None)

In [71]:
# match the currency pairs and data frame
currency_pairs_test_dict = {"AUDUSD": AUDUSD_test_df, "EURUSD": EURUSD_test_df, "GBPEUR": GBPEUR_test_df, "USDCAD": USDCAD_test_df,
                       "USDCNY": USDCNY_test_df, "USDCZK": USDCZK_test_df, "USDINR": USDINR_test_df, "USDJPY": USDJPY_test_df,
                       "USDMXN": USDMXN_test_df, "USDPLN": USDPLN_test_df}

In [72]:
# classify test data 
def classify_test_data(currency,df,column_name):
    # get original df
    train_df=currency_pairs_dict[currency]
    # sort by vol
    train_df=train_df.sort_values(by=[column_name])
    # get first bar and second bar
    first_bar_value=train_df.at[33,column_name]
    second_bar_value=train_df.at[67,column_name]
    def rule(x):
      if x<first_bar_value:
        return 1
      elif x<second_bar_value:
        return 2
      else:
        return 3
    df[column_name]=df[column_name].apply(lambda x: rule(x))

In [73]:
# classify test df
for key,df in currency_pairs_test_dict.items():
    # classify by fd
    classify_test_data(key,df,"fd")
    # classify by vol
    classify_test_data(key,df,"vol")
    # show the results to check
    currency_pairs_test_dict[key]=df

In [74]:
# extract feature to test
def extract_test_features(df):
    return df[["mean","fd","vol"]]

In [75]:
currency_perdictions_dict={}
# perdict
for key,value in currency_model_dict.items():
    test_df=extract_test_features(currency_pairs_test_dict[key])
    print(test_df)
    perdictions=regression.predict_model(value,data=test_df)
    currency_perdictions_dict[key]=perdictions

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PassiveAggressiveRegressor(C=1.0, average=False, early_stopping=False,
                           epsilon=0.1, fit_intercept=True,
                           loss='epsilon_insensitive', max_iter=1000,
                           n_iter_no_change=5, random_state=6694, shuffle=True,
                           tol=0.001, validation_fraction=0.1, verbose=0,
                           warm_start=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


       mean  fd  vol
0  0.669034   1    1
1  0.668671   3    1
2  0.668413   3    1
3  0.668596   3    1
4  0.668813   3    1
5  0.668724   3    1
6  0.668678   3    1
7  0.668940   3    1
8  0.668890   3    1
9  0.668827   3    1


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=915,
      selection='cyclic', tol=0.0001, warm_start=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


       mean  fd  vol
0  1.047952   1    1
1  1.047401   3    1
2  1.046775   3    1
3  1.047111   3    1
4  1.046890   3    1
5  1.046559   3    1
6  1.046286   3    1
7  1.046493   3    1
8  1.046464   3    1
9  1.046302   3    1


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


...

         mean  fd  vol
0  136.852707   1    1
1  136.898042   3    1
2  136.951487   3    1
3  136.936546   3    1
4  136.950411   3    1
5  137.014265   3    1
6  137.035128   3    1
7  136.939217   3    1
8  136.929442   3    1
9  136.964639   3    1


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PassiveAggressiveRegressor(C=1.0, average=False, early_stopping=False,
                           epsilon=0.1, fit_intercept=True,
                           loss='epsilon_insensitive', max_iter=1000,
                           n_iter_no_change=5, random_state=3687, shuffle=True,
                           tol=0.001, validation_fraction=0.1, verbose=0,
                           warm_start=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


        mean  fd  vol
0  19.799365   1    1
1  19.801740   3    1
2  19.797035   3    1
3  19.793685   3    1
4  19.793335   3    1
5  19.785203   3    1
6  19.791661   3    1
7  19.799066   3    1
8  19.783208   3    1
9  19.786142   3    1


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Lars(copy_X=True, eps=2.220446049250313e-16, fit_intercept=True, fit_path=True,
     jitter=None, n_nonzero_coefs=500, normalize=True, precompute='auto',
     random_state=499, verbose=False), probability_threshold=None, encoded_labels=True, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.REGRESSION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


       mean  fd  vol
0  4.481470   1    1
1  4.483005   3    1
2  4.485533   3    1
3  4.483899   3    1
4  4.485366   3    1
5  4.486296   3    1
6  4.486234   3    1
7  4.484662   3    1
8  4.485426   3    1
9  4.487500   3    1


In [88]:
# extract and join actual and perdicted return
error_dict={}
for key, df in currency_perdictions_dict.items():
    # extract label
    perdict_return=df["Label"]
    acutal_return=currency_pairs_test_dict[key]["r"]
    out=pd.concat([perdict_return, acutal_return], axis=1)
    out["error"]=out['Label']-out['r']
    small_error=np.mean(out["error"])
    error_dict[key]=small_error
    print(small_error)

3.0929054477390015e-05
0.00020280110020159335
7.827107108748922e-05
-4.238554081933999e-05
0.0
-0.00014290309333450999
4.842028597568e-05
-8.181095475099e-05
6.674531418419e-05
-0.0003042841155107092


# Helper functions

## Database initialize and reset

In [33]:
# Function slightly modified from polygon sample code to format the date string 
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')


# Function which clears the raw data tables once we have aggregated the data in a 6 minute interval
def reset_raw_data_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE " + curr[0] + curr[1] + "_raw;"))
            conn.execute(
                text("CREATE TABLE " + curr[0] + curr[1] + "_raw(ticktime text, fxrate  numeric, inserttime text);"))


# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(
                text("CREATE TABLE " + curr[0] + curr[1] + "_raw(ticktime text, fxrate  numeric, inserttime text);"))


# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database
def initialize_aggregated_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE " + curr[0] + curr[
                1] + "_agg(inserttime text, avgfxrate  numeric, stdfxrate numeric, maxfxrate numeric, minfxrate numeric，volfxrate numeric, fdfxrate numeric, r numeric);"))


def reset_aggregated_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE " + curr[0] + curr[1] + "_agg;"))
            conn.execute(text("CREATE TABLE " + curr[0] + curr[
                1] + "_agg(inserttime text, avgfxrate  numeric, stdfxrate numeric, maxfxrate numeric, minfxrate numeric, volfxrate numeric, fdfxrate numeric, r numeric);"))

def initialize_trade_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE " + curr[0] + curr[
                1] + "_trade(operation text, balance numeric, profit  numeric, status text);"))

def reset_trade_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE " + curr[0] + curr[1] + "_trade;"))
            conn.execute(text("CREATE TABLE " + curr[0] + curr[
                1] + "_trade(operation text, balance numeric, profit  numeric, status text);"))


# Perdict R

In [25]:
# classify test data 
def classify_test_data(currency,df,column_name):
    # get original df
    train_df=currency_pairs_dict[currency]
    # sort by vol
    train_df=train_df.sort_values(by=[column_name])
    # get first bar and second bar
    first_bar_value=train_df.at[33,column_name]
    second_bar_value=train_df.at[67,column_name]
    def rule(x):
      if x<first_bar_value:
        return 1
      elif x<second_bar_value:
        return 2
      else:
        return 3
    df[column_name]=df[column_name].apply(lambda x: rule(x))

In [26]:
# extract feature to test
def extract_test_features(df):
    return df[["mean","fd","vol"]]

## Agg Data

### Agg raw data

In [27]:
# This function is called every 6 minutes to aggregate the data, store it in the aggregate table,
# and then delete the raw data
def aggregate_raw_data_tables(engine, currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            result = conn.execute(text(
                "SELECT AVG(fxrate) as avg_price, COUNT(fxrate) as tot_count, MAX(fxrate) as max_price, MIN(fxrate) as min_price FROM " +
                curr[0] + curr[1] + "_raw;"))
            for row in result:
                avg_price = row.avg_price
                tot_count = row.tot_count
                max_price = row.max_price
                min_price = row.min_price
                vol_price = max_price - min_price
            std_res = conn.execute(text(
                "SELECT SUM((fxrate - " + str(avg_price) + ")*(fxrate - " + str(avg_price) + "))/(" + str(
                    tot_count) + "-1) as std_price FROM " + curr[0] + curr[1] + "_raw;"))
            for row in std_res:
                std_price = sqrt(row.std_price)
            date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM " + curr[0] + curr[1] + "_raw;"))
            for row in date_res:
                last_date = row.last_date
            # get the number of records in the dataset to know the id of current period
            period_id_data = conn.execute(text("select count(*) as num from " + curr[0] + curr[1] + "_agg;"))
            for row in period_id_data:
                period_id = row.num
            # set default value for fd
            # for the first period, we use -1 to account the cell
            fd=0
            r=0
            # at least one record in the dataset
            # so we can start to calculate Keltner Channel Upper Band and Keltner Channel Lower Band
            if period_id != 0:
                # get the latest record(=data from the previous period of 6 minutes) in agg table to fetch the data of last 6 min period
                last_period_data = conn.execute(text("select avgfxrate, volfxrate from " + curr[0] + curr[
                    1] + "_agg" + " order by inserttime desc limit 1;"))
                for row in last_period_data:
                    last_mean = row.avgfxrate
                    last_vol = row.volfxrate
                # calculate 100 Keltner Upper Bands
                upper_bound_list=[]
                lower_bound_list=[]
                for i in range(1,101):
                    upper_bound = last_mean + i * 0.025 * last_vol
                    lower_bound = last_mean - i * 0.025 * last_vol
                    upper_bound_list.append(upper_bound)
                    lower_bound_list.append(lower_bound)
                # get the number of currency pair crosses a Keltner Channel
                fxrate_data=conn.execute(text("select fxrate from "+curr[0] + curr[1]+"_raw;"))
                n=0
                sum=0
                count=0
                for row in fxrate_data:
                    sum+=row.fxrate
                    count+=1
                    for i in range(100):
                        if row.fxrate>upper_bound_list[i] or row.fxrate <lower_bound_list[i]:
                            n+=1
                # count fd
                if vol_price!=0:
                    fd=n/vol_price
                else:
                    fd=n
                # caculate return r
                cur_mean=sum/count
                r=(cur_mean-last_mean)/last_mean
            conn.execute(text("INSERT INTO " + curr[0] + curr[
                1] + "_agg (inserttime, avgfxrate, stdfxrate, maxfxrate, minfxrate, volfxrate, fdfxrate, r) VALUES (:inserttime, :avgfxrate, :stdfxrate, :maxfxrate, :minfxrate, :volfxrate, :fdfxrate, :r);"),
                         [{"inserttime": last_date, "avgfxrate": avg_price, "stdfxrate": std_price,
                           "maxfxrate": max_price, "minfxrate": min_price, "volfxrate": vol_price, "fdfxrate":fd,"r":r}])

            # This calculates and stores the return values
            exec("curr[2].append(" + curr[0] + curr[1] + "_return(last_date,avg_price))")
            #exec("print(\"The return for "+curr[0]+curr[1]+" is:"+str(curr[2][-1].hist_return)+" \")")

            if len(curr[2]) > 5:
                try:
                    avg_pop_value = curr[2][-6].hist_return
                except:
                    avg_pop_value = 0
                if isnan(avg_pop_value) == True:
                    avg_pop_value = 0
            else:
                avg_pop_value = 0
            # Calculate the average return value and print it/store it
            curr_avg = curr[2][-1].get_avg(avg_pop_value)
            #exec("print(\"The average return for "+curr[0]+curr[1]+" is:"+str(curr_avg)+" \")")

            # Now that we have the average return, loop through the last 5 rows in the list to start compiling the 
            # data needed to calculate the standard deviation
            for row in curr[2][-5:]:
                row.add_to_running_squared_sum(curr_avg)

            # Calculate the standard dev using the avg
            curr_std = curr[2][-1].get_std()
            #exec("print(\"The standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_std)+" \")")

            # Calculate the average standard dev
            if len(curr[2]) > 5:
                try:
                    pop_value = curr[2][-6].std_return
                except:
                    pop_value = 0
            else:
                pop_value = 0
            curr_avg_std = curr[2][-1].get_avg_std(pop_value)
            #exec("print(\"The average standard deviation of the return for "+curr[0]+curr[1]+" is:"+str(curr_avg_std)+" \")")

            # -------------------Investment Strategy-----------------------------------------------
            try:
                return_value = curr[2][-1].hist_return
            except:
                return_value = 0
            if isnan(return_value) == True:
                return_value = 0

            try:
                return_value_1 = curr[2][-2].hist_return
            except:
                return_value_1 = 0
            if isnan(return_value_1) == True:
                return_value_1 = 0

            try:
                return_value_2 = curr[2][-3].hist_return
            except:
                return_value_2 = 0
            if isnan(return_value_2) == True:
                return_value_2 = 0

            try:
                upp_band = curr[2][-1].avg_return + (1.5 * curr[2][-1].std_return)
                if return_value >= upp_band and curr[
                    3].Prev_Action_was_Buy == True and return_value != 0:  #  (return_value > 0) and (return_value_1 > 0) and
                    curr[3].sell_curr(avg_price)
            except:
                pass

            try:
                loww_band = curr[2][-1].avg_return - (1.5 * curr[2][-1].std_return)
                if return_value <= loww_band and curr[
                    3].Prev_Action_was_Buy == False and return_value != 0:  # and  (return_value < 0) and (return_value_1 < 0)
                    curr[3].buy_curr(avg_price)
            except:
                pass


### Agg trade data

In [89]:
# currency pair
# input current time period
def set_trade_data_tables(engine, currency_pairs, period):
    with engine.begin() as conn:
        for curr in currency_pairs[:5]:
            lost_list=[-0.0025, -0.0015, -0.001, -0.0005, -0.0005, -0.0005,-0.0005, -0.0005, -0.0005, -0.0005]
            # get the sum of last 10 period r data
            data=conn.execute(text("SELECT avgfxrate as mean, volfxrate as vol, fdfxrate as fd, r as r FROM "+curr[0]+curr[1]+"_agg order by inserttime limit 10;"))
            r_list=[]
            # gather data used to perdict
            data_matrix=[]
            for row in data:
              r_list.append(row.r)
              line=[]
              line.append(row.mean)
              line.append(row.vol)
              line.append(row.fd)
              data_matrix.append(line)
            sum_r=0
            for item in r_list:
              sum_r+=item
            # perdict r value for each records
            df=pd.DataFrame(data_matrix)
            names=["mean","vol","fd"]
            df.columns=names
            cur_model_key=""+curr[0]+curr[1]
            classify_test_data(cur_model_key,df,"vol")
            classify_test_data(cur_model_key,df,"fd")
            df=extract_test_features(df)
            # perdict
            perdictions=regression.predict_model(currency_model_dict[cur_model_key],data=df)
            # sum perdicted r
            sum_perdicted=np.sum(perdictions)
            sum_perdicted_r=sum_perdicted[-1]
            print(sum_perdicted_r)
            # get balance
            # default balance=100
            balance=100
            if sum_r!=0:
                # update balance
                balance_data=conn.execute(text("SELECT balance FROM "+curr[0]+curr[1]+"_trade where operation=\'long\';"))
                for row in balance_data:
                    balance=row.balance
            if sum_r>lost_list[period-1] and sum_perdicted_r>lost_list[period-1] and abs(sum_r-sum_perdicted_r)<abs(error_dict[cur_model_key]):
                if period<=4:
                    # calculate profit
                    profit=balance+sum_r
                    balance+=profit+100
                    # update data in table
                    conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"long",'balance': balance, 'profit': profit, 'status':'reinvest'}])
                else:
                    # calculate profit
                    profit=balance+sum_r
                    balance+=profit
                    # update data in table
                    conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"long",'balance': balance, 'profit': profit, 'status':'reinvest'}])
            elif sum_r>lost_list[period-1] and sum_perdicted_r<lost_list[period-1] or sum_r<lost_list[period-1] and sum_perdicted_r>lost_list[period-1]:
                # calculate profit
                profit=balance+sum_r
                balance+=profit
                conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"long",'balance': balance, 'profit': profit, 'status':'stop'}])
            else:
                # calculate profit
                profit=balance+sum_r
                balance+=profit
                conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"long",'balance': balance, 'profit': profit, 'status':'closed'}])
        for curr in currency_pairs[5:]:
            lost_list=[0.0025, 0.0015, 0.001, 0.0005, 0.0005, 0.0005,0.0005, 0.0005, 0.0005, 0.0005]
            # get the sum of last 10 period r data
            r_data=conn.execute(text("SELECT sum(r) as sum_r FROM "+curr[0]+curr[1]+"_agg order by inserttime limit 10;"))
            sum_r=0
            for row in r_data:
                sum_r=row.sum_r
            # get balance
            balance=100
            if sum_r!=0:
                # update balance
                balance_data=conn.execute(text("SELECT balance FROM "+curr[0]+curr[1]+"_trade where operation=\'short\';"))
                for row in balance_data:
                    balance=row.balance
            if sum_r>lost_list[period-1] and sum_perdicted_r>lost_list[period-1] and abs(sum_r-sum_perdicted_r)<abs(error_dict[cur_model_key]):
                if period<=4:
                    # calculate profit
                    profit=balance+sum_r*(-1)
                    balance+=profit+100
                    # update data in table
                    conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"short",'balance': balance, 'profit': profit, 'status':'open'}])
                else:
                    # calculate profit
                    profit=balance+sum_r*(-1)
                    balance+=profit
                    # update data in table
                    conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"short",'balance': balance, 'profit': profit, 'status':'open'}])
            elif sum_r>lost_list[period-1] and sum_perdicted_r<lost_list[period-1] or sum_r<lost_list[period-1] and sum_perdicted_r>lost_list[period-1]:
                # calculate profit
                profit=balance+sum_r
                balance+=profit
                conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"short",'balance': balance, 'profit': profit, 'status':'stop'}])
            else:
                # calculate profit
                profit=balance+sum_r*(-1)
                balance+=profit
                conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_trade VALUES (:operation, :balance, :profit, :status);"), [{'operation':"short",'balance': balance, 'profit': profit, 'status':'closed'}])

# Execute Main Function

In [29]:
%load_ext sql

In [30]:
%%sql
sqlite:///final.db

'Connected: @final.db'

In [ ]:
# This main function repeatedly calls the polygon api every 1 seconds for 24 hours 
# and stores the results.
def main(currency_pairs):
    
    key = "beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq"

    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    total_count = 0

    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite:///final.db", echo=False, future=True)

    # # Create the needed tables in the database
    # reset_raw_data_tables(engine, currency_pairs)
    # reset_aggregated_tables(engine, currency_pairs)

    # # create trade tables
    # reset_trade_tables(engine, currency_pairs)
    # Create the needed tables in the database
    reset_raw_data_tables(engine, currency_pairs)
    reset_aggregated_tables(engine, currency_pairs)

    # create trade tables
    reset_trade_tables(engine, currency_pairs)

    # Open a RESTClient for making the api calls
    # with RESTClient(key) as client:
    try:
        client=RESTClient(key)
        # Loop that runs until the total duration of the program hits 24 hours. 
        while count < 86400:  # 86400 seconds = 24 hours

            # Make a check to see if 6 minutes has been reached or not
            if agg_count == 10:
                # Aggregate the data and clear the raw data tables
                aggregate_raw_data_tables(engine, currency_pairs)
                reset_raw_data_tables(engine, currency_pairs)
                agg_count = 0

            if total_count>=100 and total_count%100==0:
                set_trade_data_tables(engine, currency_pairs, int(total_count/100))
            # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
            # code takes about .15 seconds to run
            time.sleep(0.75)

            # Increment the counters
            count += 1
            agg_count += 1
            total_count += 1

            # Loop through each currency pair
            for currency in currency_pairs:
                # Set the input variables to the API
                from_ = currency[0]
                to = currency[1]

                # Call the API with the required parameters
                try:
                    resp = client.get_real_time_currency_conversion(from_, to, amount=100, precision=2)
                except:
                    continue

                # This gets the Last Trade object defined in the API Resource
                last_trade = resp.last

                # Format the timestamp from the result
                dt = ts_to_datetime(last_trade.timestamp)

                # Get the current time and format it
                insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                # Calculate the price by taking the average of the bid and ask prices
                avg_price = (last_trade.bid + last_trade.ask) / 2

                # Write the data to the SQLite database, raw data tables
                with engine.begin() as conn:
                    conn.execute(text(
                        "INSERT INTO " + from_ + to + "_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),
                        [{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])
    finally:
        pass

# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["AUD", "USD", [], portfolio("AUD", "USD")],
                  ["GBP", "EUR", [], portfolio("GBP", "EUR")],
                  ["USD", "CAD", [], portfolio("USD", "CAD")],
                  ["USD", "JPY", [], portfolio("USD", "JPY")],
                  ["USD", "MXN", [], portfolio("USD", "MXN")],
                  ["EUR", "USD", [], portfolio("EUR", "USD")],
                  ["USD", "CNY", [], portfolio("USD", "CNY")],
                  ["USD", "CZK", [], portfolio("USD", "CZK")],
                  ["USD", "PLN", [], portfolio("USD", "PLN")],
                  ["USD", "INR", [], portfolio("USD", "INR")]]

# Run the main data collection loop
main(currency_pairs)